# MedGemma Trauma Analysis — Colab Test Runner
**Branch:** `feature/deep-volume-qa`

Run cells top-to-bottom. Steps:
1. Install dependencies
2. Clone repo & switch to feature branch
3. Set tokens
4. Export real RSNA trauma CT slices for testing
5. (Optional) Clear GPU memory
6. Launch Flask app via ngrok

After launching, open the ngrok URL and upload files from `/content/sample_ct/` to test.

In [ ]:
# Cell 1 — Install dependencies
# datasets must be pinned to <3.0.0 — newer versions dropped loading script support
# which breaks the RSNA dataset used by export_sample_images.py
!pip install -q \
    transformers>=4.47.0 \
    peft>=0.10.0 \
    bitsandbytes \
    accelerate \
    flask \
    flask-cors \
    pyngrok \
    nibabel \
    pydicom \
    segmentation-models-pytorch \
    sentencepiece \
    protobuf \
    "datasets==2.21.0"

In [ ]:
# Cell 2 — Clone repo & switch to feature branch
!git clone https://github.com/AryanMarwah7781/medgemma-trauma-analysis
%cd medgemma-trauma-analysis
!git fetch origin
!git switch feature/deep-volume-qa
!git pull origin feature/deep-volume-qa
!ls

In [ ]:
# Cell 3 — Set tokens (replace placeholders with your actual tokens)
import os

os.environ["HF_TOKEN"]     = "hf_YOUR_TOKEN_HERE"        # huggingface.co/settings/tokens
os.environ["NGROK_TOKEN"]  = "YOUR_NGROK_TOKEN_HERE"     # dashboard.ngrok.com/authtokens
os.environ["USE_NGROK"]    = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Optional: point to a saved LoRA adapter on Drive
# os.environ["LORA_ADAPTER"] = "/content/drive/MyDrive/medgemma_lora/final_adapter"

print("Tokens set. HF_TOKEN starts with:", os.environ["HF_TOKEN"][:8])

In [ ]:
# Cell 4 — Export real labeled RSNA trauma CT slices
# Pulls actual abdominal trauma CT volumes (liver, spleen, kidney injury + extravasation)
# and saves them as PNG files ready to upload to the web UI.
# Requires: HF_TOKEN set in Cell 3, and HuggingFace dataset terms accepted at
#   https://huggingface.co/datasets/jherng/rsna-2023-abdominal-trauma-detection

!python scripts/export_sample_images.py \
    --out_dir /content/sample_ct \
    --n 10 \
    --hf_token $HF_TOKEN

# After this runs, check the manifest to see what injury types were exported:
!cat /content/sample_ct/manifest.txt

In [ ]:
# Cell 5 — Clear GPU memory before launching (run if needed)
import gc, torch
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
print(f"Free VRAM: {torch.cuda.mem_get_info()[0] / 1e9:.1f} GB")

In [ ]:
# Cell 6 — Launch Flask app with ngrok tunnel
# The public URL will be printed below — open it in your browser
!python app.py